In [ ]:
# Define Enum for LLMProvider Options
from enum import Enum
class LLMProvider(Enum):
    OPENAI = "openai"
    GEMINI = "gemini"
    ANTHROPIC = "anthropic"

In [ ]:
# Define Base Config which will be implemented by model specific configs
from pydantic import BaseModel, ConfigDict
from typing import Optional

class BaseAgentConfig(BaseModel):
    model_config = ConfigDict(extra="forbid")
    model: str
    temperature: Optional[float] = None
    max_tokens: Optional[int] = None

In [ ]:
# Provider Specific Config Extensions
from typing import Dict, Any

class OpenAIConfig(BaseAgentConfig):
    top_p: Optional[float] = None
    presence_penalty: Optional[float] = None
    frequency_penalty: Optional[float] = None


class GeminiConfig(BaseAgentConfig):
    safety_settings: Optional[Dict[str, Any]] = None
    top_k: Optional[int] = None

In [ ]:
# Model for messages
from typing import List, Literal
from pydantic import BaseModel

class Message(BaseModel):
    role: Literal["system", "user", "assistant", "tool"]
    content: str

In [ ]:
# Base Provider class (llm-provider)
from abc import ABC, abstractmethod
from typing import Generator, Dict, Any, List, Type

class LLMProvider(ABC):
    config_schema: Type[BaseAgentConfig]

    def __init__(self, config: BaseAgentConfig, system_message: str):
        self.config = config
        self.system_message = system_message
        
        self.history: List[Message] = [
            Message(role="system", content=system_message)
        ]

    # -------------------------
    # History Management
    # -------------------------
    def add_user_message(self, content: str):
        self.history.append(Message(role="user", content=content))

    def add_assistant_message(self, content: str):
        self.history.append(Message(role="assistant", content=content))

    def rollback(self, steps: int = 1):
        """Go back N messages (excluding system)."""
        if steps > 0:
            self.history = self.history[:-steps]

    def save_history(self) -> List[Message]:
        return self.history.copy()

    def compress_history(self, summarizer: "LLMProvider"):
        """Replace history with a summary."""
        summary = summarizer.generate(
            "Summarize the following conversation:\n"
            + "\n".join(m.content for m in self.history)
        )
        self.history = [
            Message(role="system", content=self.system_message),
            Message(role="assistant", content=summary)
        ]

    # -------------------------
    # Generation APIs
    # -------------------------
    @abstractmethod
    def generate(self, prompt: str, **kwargs) -> str:
        pass

    @abstractmethod
    def generate_stream(self, prompt: str, **kwargs) -> Generator[str, None, None]:
        pass

    @abstractmethod
    def generate_structured(
        self,
        prompt: str,
        response_schema: Dict[str, Any],
        **kwargs
    ) -> Dict[str, Any]:
        pass

    @abstractmethod
    def generate_tool_calls(
        self,
        prompt: str,
        tools: List[BaseModel],
        stream: bool = False,
        **kwargs
    ):
        pass

    # -------------------------
    # Introspection
    # -------------------------
    @classmethod
    def supported_config(cls) -> Dict[str, Any]:
        """Expose valid config fields to user/dev."""
        return cls.config_schema.model_json_schema()
